In [1]:
import numpy as np
import cv2

In [2]:
# Retrieved example header from http://paulbourke.net/dataformats/ply/example1.ply

# The PLY format encodes an object as a collection of vertices, faces and other elements, and other 
# important properties like oclor and normal direction. Some of the objects include:
# polygon objects from modeling programs, range data, terrain data, and more. 
header = '''ply
                format ascii 1.0
                element vertex %(vert_num)d
                property float x
                property float y
                property float z
                property uchar red
                property uchar green
                property uchar blue
                end_header
                '''

In [1]:
# Code for exporting the 3d Image to a 3rd party readable format. What the format does is explained above.
# From https://stackoverflow.com/questions/46959106/f-write-unsupported-format-character-error-after-changing-input-data
def write_ply(fn, verts, colors):
    verts = verts.reshape(-1, 3)
    colors = colors.reshape(-1, 3)
    verts = np.hstack([verts, colors])
    with open(fn, 'wb') as f:
        f.write((header % dict(vert_num=len(verts))).encode('utf-8'))
        np.savetxt(f, verts, fmt='%f %f %f %d %d %d ')

In [5]:
imgL = cv2.imread('left_aloe.jpg')
imgR = cv2.imread('right_aloe.jpg')

# Downscale images for faster processing
# We lose pixel information when we downscale, but we chose to downscale anyways for faster computation.
imgL = cv2.pyrDown(imgL)
imgR = cv2.pyrDown(imgR)

# Parameters to determine the quality of the disparity map
# window: size of the window or the size of the feature matching part
# mdisp: minimum disparity as a limit
# ndisp: number of disparity as a tolerance
window = 3
mdisp = -1
ndisp = 16

# https://docs.opencv2.org/3.4/d2/d85/classcv2_1_1StereoSGBM.html
# Some parameters are changed due to different image input
stereo = cv2.StereoSGBM_create(minDisparity = mdisp,
    numDisparities = ndisp,
    blockSize = 15,
    P1 = 8 * 3 * window ** 2,
    P2 = 32 * 3 * window ** 2,
    disp12MaxDiff = 1,
    uniquenessRatio = 10,
    speckleWindowSize = 5,
    speckleRange = 5,
)

# Using cv2's method to compute the disparity of the image.
disparity = stereo.compute(imgL, imgR).astype(np.float32) / (ndisp * 1.0)

# From https://ags.cs.uni-kl.de/fileadmin/inf_ags/3dcv-ws14-15/3DCV_lec01_camera.pdf
# reprojecting the image, and flipping the image based on y
Q = np.float32([[1, 0, 0, -0.5 * imgL.shape[:2][1]],
                [0,-1, 0, 0.5 * imgL.shape[:2][0]],
                [0, 0, 0, -0.8 * imgL.shape[:2][1]],
                [0, 0, 1, 0]])

# reprojecting
pts = cv2.reprojectImageTo3D(disparity, Q)
colors = cv2.cvtColor(imgL, cv2.COLOR_BGR2RGB)

m = disparity > disparity.min()
write_ply('output.ply', pts[m], colors[m])

loading images...
computing disparity...
generating 3d point cloud...
out.ply saved
Done
